In [1]:
import dash
from jupyter_dash import JupyterDash
from dash import Dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import plotly.express as px
import datetime as dt
import time
import numpy as np

In [2]:
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'font-size': '30px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#000091',
    'color': 'white',
    'padding': '6px',
    'font-size': '30px'
}

header = {
    'font-size': '25px',
    'fontWeight': 'bold'
}

searchBox = {
    'width': '100%',
    'height': '200px',
    'margin-top': '-802px',
    'margin-left': '320px',
    'borderWidth': '1px',
    'borderStyle': 'solid',
    'borderRadius': '5px',
    'borderColor': '#000091',
    'padding-left': '10px'
}

In [3]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.H1(children=['Rent-A-Bike DBMS'], style=header),
    html.Div(id='home', className='control_tabs',children=[
        dcc.Tabs(id='view_tabs',value='bookings', children=[
            dcc.Tab(
                label='Bookings',
                value='bookings',
                style=tab_style,
                selected_style=tab_selected_style
            ),
            dcc.Tab(
                label='HR',
                value='hr',
                style=tab_style,
                selected_style=tab_selected_style
            )
        ])
    ])
])

In [6]:
if __name__ == '__main__':
    app.run_server(debug=False, mode='external', host='127.0.0.1',port=8000)

 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jun/2022 10:31:57] "GET /_alive_c99aaa73-1fcc-44be-8c92-e3d2094dd6b2 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8000/
